# Estimating Gender Discrimination in the Workplace
In this exercise we’ll use data from the 2018 US Current Population Survey (CPS) to try and estimate the effect of being a woman on workplace compensation. Note that our focus will be only on differential compensation in the work place, and as a result it is important to bear in mind that our estimates are not estimates of all forms of gender discrimination. For example, these analyses will not account for things like gender discrimination in terms of getting jobs.

## Exercise 1:
Begin by downloading and importing 2018 CPS data from http://www.github.com/nickeubank/MIDS_Data/Current_Population_Survey. The file is called morg18.dta and is a Stata dataset. Additional data on the dataset can be found by following the links in the README.txt file in the folder, but for the moment it is sufficient to know this is a national survey run in the United States.

The survey does include some survey weights we won’t be using (i.e. not everyone in the sample was included with the same probability), so the numbers we estimate will not be perfect estimates of the gender wage gap in the United States, but they are pretty close.

In [60]:
import pandas as pd

# Load survey
df = pd.read_stata('./data/morg18.dta')
df.head()

,hhid,intmonth,hurespli,hrhtype,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,...,ym_file,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02
0,000004795110719,January,1.0,Husband/wife primary fam (neither in Armed For...,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0601,06011,1,1,...,696,681,0,0,0,0,0,14.0,NaN,NaN
1,000004795110719,January,1.0,Husband/wife primary fam (neither in Armed For...,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0601,06011,1,1,...,696,681,0,0,0,0,0,13.0,NaN,NaN
2,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,696,693,0,0,0,1,0,12.0,Office and administrative support occupations,"Health care services , except hospitals"
3,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,696,693,0,0,0,0,0,12.0,Office and administrative support occupations,Administrative and support services
4,000110359424339,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0711,07111,1,1,...,696,693,0,0,0,0,0,NaN,Healthcare practitioner and technical occupations,Hospitals


## Exercise 2
Because our interest is only in-the-workplace wage discrimination among full-time workers, we need to start by subsetting our data for people currently employed at the time of this survey using the lfsr94 variable, who are employed full time (meaning that their usual hours per week – uhourse – is 35 or above).

As noted above, this analysis will miss many forms of gender discrimination. For example, in dropping anyone who isn’t working, we immediately lose any women who couldn’t get jobs, or who chose to lose the workforce because the wages they were offered (which were likely lower than those offered men) were lower than they were willing / could accept. And in focusing on full time employees, we miss the fact women may not be offered full time jobs at the same rate as men.

In [61]:
df = df[df.lfsr94 == 'Employed-At Work']
df = df[df.uhourse >= 35]

df.shape

(133814, 98)

## Exercise 3
Now let’s estimate the basic wage gap for the United States!

Earnings per hour worked can be found in the earnhre variable. Two things are worth noting about this variable:

- It is coded in cents (1/100 of a dollar), not dollars, so make sure to divide by 100 to get dollars.

- Earnings are “top-coded” at 9999 (meaning any value above 99.99 dollars an hour is coded as 99.99 dollars an hour). Thankfully these are rare, so we’ll just leave them in as-is for now. However, note that wage inequality is likely to be especially high for extremely high paid individuals (e.g. most CEOs are men), so this will bias us towards slightly conservative (low) estimates of the gender wage gap.

Using the variable sex (1=Male, 2=Female), estimate the gender wage gap in terms of wages per hour worked!

In [63]:
# Adjust earnings per hour (in cents) into dollars,
df['earnhre_dollars'] = df['earnhre'] / 100

import statsmodels.formula.api as smf

df['female'] = (df.sex == 2).astype('int')
smf.ols('earnhre_dollars ~ female', df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        earnhre_dollars   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     999.7
Date:                Fri, 19 Feb 2021   Prob (F-statistic):          8.84e-218
Time:                        21:20:32   Log-Likelihood:            -2.4478e+05
No. Observations:               65755   AIC:                         4.896e+05
Df Residuals:                   65753   BIC:                         4.896e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     20.5546      0.053    386.055      0.000      20.450      20.659
female        -2.4759      0.078    -31.619      0.000      -2.629      -2.322
==============================================================================
Omnibus:                    31043.263   Durbin-Watson:                   1.852
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           210689.419
Skew:                           2.182   Prob(JB):                         0.00
Kurtosis:                      10.606   Cond. No.                         2.54
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

According to the OLS model, there is a gender wage gap. The coefficient shows that females' per hour worked income is significantly lower than the male by -2.47 dollars. According to the p-value, the difference is statistically significant.

## Exercise 4
The variable uhourse is the number of hours that the respondent usually works per week. What is the wage gap not per hour, but per year? Is the difference statistically significant?

In [64]:
df['annual_earnings'] = df['earnhre_dollars'] * df['uhourse'] * 52

smf.ols('annual_earnings ~ female', df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     1637.
Date:                Fri, 19 Feb 2021   Prob (F-statistic):               0.00
Time:                        21:20:37   Log-Likelihood:            -7.5335e+05
No. Observations:               65755   AIC:                         1.507e+06
Df Residuals:                   65753   BIC:                         1.507e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4.511e+04    121.688    370.664      0.000    4.49e+04    4.53e+04
female     -7240.6673    178.971    -40.457      0.000   -7591.450   -6889.884
==============================================================================
Omnibus:                    33612.087   Durbin-Watson:                   1.863
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           291856.174
Skew:                           2.309   Prob(JB):                         0.00
Kurtosis:                      12.230   Cond. No.                         2.54
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Answer：**
The salary gap still exists in annual income; females' yearly salary is significantly lower than the male by 7240 dollars. According to the p-value, the difference is statistically significant.

## Exercise 5
We just compared all full-time working men to all full-time working women. For this to be an accurate causal estimate of the effect of being a woman in the work place, what must be true of these two groups? What is one reason that this may not be true?

**Answer：** Age is a potential factor may affect the comparison, because veterans tend to receive more salaries than a rookie. To get an accurate causal estimate, we must eliminate this difference from age. The difference may exist because women workers may return home after having children, which will bring down the average working years of female works.

## Exercise 6
One answer to the second part of Exercise 5 is that working women are likely to be younger, since a larger portion of younger women are entering the workforce as compared to older generations.

**Answer：** To control for this difference, let’s now regress annual earnings on gender and age. What is the implied average annual wage difference between women and men? Is it different from your raw estimate? Is the difference statistically significant?

In [16]:
smf.ols('annual_earnings ~ female + age', df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     1974.
Date:                Fri, 19 Feb 2021   Prob (F-statistic):               0.00
Time:                        17:07:18   Log-Likelihood:            -7.5225e+05
No. Observations:               65755   AIC:                         1.504e+06
Df Residuals:                   65752   BIC:                         1.505e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.265e+04    288.243    113.284      0.000    3.21e+04    3.32e+04
female     -7557.2936    176.107    -42.913      0.000   -7902.462   -7212.125
age          304.1829      6.406     47.485      0.000     291.627     316.738
==============================================================================
Omnibus:                    33771.778   Durbin-Watson:                   1.868
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           307310.621
Skew:                           2.305   Prob(JB):                         0.00
Kurtosis:                      12.535   Cond. No.                         146.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Answer：** The implied average annual wage difference between women and men is 7557 dollars. Since the p-value is less than 0.001, the result is statistically significant. The difference is larger than the raw estimate (7240 dollars).

## Exercise 7
In running this regression and interpreting the coefficient on female, what is the implicit comparison you are making? In other words, when we run this regression and interpreting the coefficient on female, we’re basically pretending we are comparing two groups and assuming they are counter-factuals for one another. What are these two groups?

**Answer：**   We are interpreting the implicit comparison of women versus men in the model.  Strictly speaking, it should be female and non-female. Still, in this dataset, the sex item only has two alternatives: men and women. 

# Exercise 8
Now let’s add to our regression an indicator variable for whether the respondent has at least graduated high school, and an indicator for whether the respondent at least has a BA.

In answering this question, use the following table of codes for the variable grade92.

Education is coded as follows:

![cps_educ_codes](./img/cps_educ_codes.png)  

In [66]:
df['high_school'] = (df.grade92 >= 39).astype('int')
df['ba'] = (df.grade92 >= 42).astype('int')

print("Number of high_school is {}, BA is {}.".
      format(sum(df['high_school']), sum(df['ba'])))

Number of high_school is 125640, BA is 62617.


In [67]:
smf.ols('annual_earnings ~ female + age + high_school + ba', df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     2786.
Date:                Fri, 19 Feb 2021   Prob (F-statistic):               0.00
Time:                        21:21:06   Log-Likelihood:            -7.4902e+05
No. Observations:               65755   AIC:                         1.498e+06
Df Residuals:                   65750   BIC:                         1.498e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    2.139e+04    381.994     56.003      0.000    2.06e+04    2.21e+04
female      -9432.7984    169.206    -55.747      0.000   -9764.443   -9101.154
age           306.2583      6.101     50.197      0.000     294.300     318.217
high_school  9121.8502    296.897     30.724      0.000    8539.932    9703.768
ba           1.312e+04    189.998     69.056      0.000    1.27e+04    1.35e+04
==============================================================================
Omnibus:                    31930.498   Durbin-Watson:                   1.896
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           289837.885
Skew:                           2.144   Prob(JB):                         0.00
Kurtosis:                      12.349   Cond. No.                         234.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Answer：**  This model shows that having at least a high_school degree would increase the salary by 9121 dollars. At least a bachelor's degree would increase the wage by 13120 dollars. The two effects are all statistically significant. However, females' annual salary is significantly lower than men by 9432 dollars, which is even worse than the previous model's values.  

## Exercise 9
In running this regression and interpreting the coefficient on female, what is the implicit comparison you are making? In other words, when we run this regression and interpreting the coefficient on female, we are once more basically pretending we are comparing two groups and assuming they are counter-factuals for one another. What are these two groups?

**Answer：** We assume that these difference are equal across all industries which is ounter-factuals. 

## Exercise 10
Given how the coefficient on female has changed between Exercise 6 and Exercise 8, what can you infer about the educational attainment of the women in your survey data (as compared to the educational attainment of men)?

In [68]:

print("Number of male high_school is {}, BA is {}.".
      format(sum(df[df["female"] == 0]['high_school']), sum(df[df["female"] == 0]['ba'])))

print("Number of female high_school is {}, BA is {}.".
      format(sum(df[df["female"] == 1]['high_school']), sum(df[df["female"] == 1]['ba'])))

Number of male high_school is 69353, BA is 31955.
Number of female high_school is 56287, BA is 30662.


## Exercise 11
**Answer：** What does that tell you about the potential outcomes of men and women before you added education as a control?

## Exercise 12
Finally, let’s include fixed effects for the type of job held by each respondent.

Fixed effects are a method used when we have a nested data structure in which respondents belong to groups, and those groups may all be subject to different pressures. In this context, for example, we can add fixed effects for the industry of each respondent – since wages often vary across industries, controlling for industry is likely to improve our estimates.

(Note that fixed effects are very similar in principle to hierarchical models. There are some differences you will read about for our next class, but they are designed to serve the same role, just with slightly different mechanics).

When we add fixed effects for groups like this, our interpretation of the other coefficients changes. Whereas in previous exercises we were trying to explain variation in men and women’s wages across all respondents, we are now effectively comparing men and women’s wages within each employment sector. Our coefficient on female, in other words, now tells us how much less (on average) we would expect a woman to be paid than a man within the same industry, not across all respondents.

(Note that running this regression will result in lots of coefficients popping up you don’t care about. We’ll introduce some more efficient methods for adding fixed effects that aren’t so messy in a later class – for now, you can ignore those coefficients!)

In [69]:
# from linearmodels import PanelOLS
# 
# df_multiindex = df.set_index(['ind02', df.index])
# 
# mod = PanelOLS.from_formula('annual_earnings ~ female + age + high_school + ba + EntityEffects',
#                             data=df_multiindex)
# mod.fit(cov_type='clustered', cluster_entity=True)

In [71]:
df.reset_index(drop=True, inplace=True)
smf.ols('annual_earnings ~ female + age + high_school + ba + C(ind02)', df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     74.49
Date:                Fri, 19 Feb 2021   Prob (F-statistic):               0.00
Time:                        21:24:48   Log-Likelihood:            -7.4559e+05
No. Observations:               65755   AIC:                         1.492e+06
Df Residuals:                   65492   BIC:                         1.494e+06
Df Model:                         262                                         
Covariance Type:            nonrobust                                         
============================================================================================================================================================================================
                                                                                                                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                  1.91e+04   1045.590     18.271      0.000    1.71e+04    2.12e+04
C(ind02)[T.Animal production (112)]                                                                                       1633.5560   1639.424      0.996      0.319   -1579.715    4846.827
C(ind02)[T.Forestry except logging (1131, 1132)]                                                                          6797.9723   3455.663      1.967      0.049      24.872    1.36e+04
C(ind02)[T.Logging (1133)]                                                                                                6352.4705   2901.157      2.190      0.029     666.203     1.2e+04
C(ind02)[T.Fishing, hunting, and trapping (114)]                                                                         -5132.5912   5190.582     -0.989      0.323   -1.53e+04    5040.950
C(ind02)[T.Support activities for agriculture and forestry (115)]                                                         2558.4307   2632.405      0.972      0.331   -2601.083    7717.945
C(ind02)[T.Oil and gas extraction (211)]                                                                                  2.497e+04   3413.239      7.315      0.000    1.83e+04    3.17e+04
C(ind02)[T.Coal mining (2121)]                                                                                            2.453e+04   2219.156     11.056      0.000    2.02e+04    2.89e+04
C(ind02)[T.Metal ore mining (2122)]                                                                                       3.836e+04   3374.951     11.368      0.000    3.17e+04     4.5e+04
C(ind02)[T.Nonmetallic mineral mining and quarrying (2123)]                                                               1.629e+04   2050.355      7.943      0.000    1.23e+04    2.03e+04
C(ind02)[T.Support activities for mining (213)]                                                                           2.702e+04   1436.064     18.815      0.000    2.42e+04    2.98e+04
C(ind02)[T.Electric power generation, transmission and distribution (Pt. 2211)]                                           2.191e+04   1397.395     15.679      0.000    1.92e+04    2.46e+04
C(ind02)[T.Natural gas distribution (Pt.s2212)]                                                                           1.572e+04   2416.026      6.508      0.000     1.1e+04    2.05e+04
C(ind02)[T.Electric and gas, and other combinations (Pts. 2

**Answer：** According to the model, women will receive 7319 dollars less than men in the same industry. The result is statistically significant.

In [56]:
pd.Series(df["ind02"].unique()).to_csv("ind.csv")

## Exercise 13
Now that we’ve added industry fixed effects, what groups are we implicitly treated as counter-factuals for one another now?

**Answer：** The industry of "Crop production (111)" is  implicitly treated as counter-factuals.

## Exercise 14
What happened to your estimate of the gender wage gap when you added industry fixed effects? What does that tell you about the industries chosen by women as opposed to men?

**Answer：** 